<a href="https://colab.research.google.com/github/Mobeenahmedmehr/Mobeen_Ahmed/blob/main/video_and_audio_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing Google_genai**

In [ ]:
!pip install --upgrade --quiet google-genai

# **Importing API Key**

In [ ]:
from google.colab import userdata
GEMINI_API_KEY: str = userdata.get('GOOGLE_API_KEY')
if(GEMINI_API_KEY):
  print("API Key found")
else:
  print("API Key not found")

API Key found


In [ ]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"


# **Uplaoding a video**

In [ ]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  print(f"Uploaded file:{filename}")

Saving introduction_video.mp4 to introduction_video.mp4
Uploaded file:introduction_video.mp4


# **Video understanding by Gemini Flash 2.0 exp**

In [ ]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path="introduction_video.mp4")
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

introduction_video = upload_video('introduction_video.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/hg01ch9st6g9


In [ ]:
from google.genai.types import Content, Part
from IPython.display import Markdown
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = introduction_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                     file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)

Markdown(response.text)


```json
[
  {
    "timecode": "00:00",
    "caption": "A man with dark hair is centered in the frame and speaking. He is wearing a gray jacket and a dark shirt. Behind him is a wall and a small shelf with pens."
   },
  {
    "timecode": "00:01",
        "caption": "The man speaks, “Assalamualaikum, mera naam Mubeen Ahmed hai.”"
  },
 {
    "timecode": "00:06",
        "caption": "The man continues speaking, “Mai AI engineering kar raha hu PIAIC say.”"
    }
]
```
